In [1]:
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import json
import pprint
from pymongo import MongoClient

In [3]:
year = ['2016', '2017', '2018', '2019']

df_list = {}
for y in year:
    file_name = f"../Data/SF_Sample_{y}.csv"
    df_list[y] = pd.read_csv(file_name, usecols=['sourceid', 'dstid', 'hod', 'mean_travel_time', 'standard_deviation_travel_time'])
    

In [5]:
# for y in year:
#     print(df_list[y].head())
#     print('------------------------------')

In [7]:
engine = create_engine("sqlite:///../Data/files.sqlite")
conn = engine.connect()
Base = automap_base()
Base.prepare(engine, reflect=True)
for y in year:
    df_list[y].to_sql(y,conn, if_exists='replace', index = False)

In [8]:
inspector = inspect(engine)
inspector.get_table_names()

['2016', '2017', '2018', '2019']

In [18]:
path = os.path.join('..','Data', 'SF_Features.json')
with open(path) as json_file:
    data = json.load(json_file)

In [19]:
client = MongoClient('localhost', 27017)
db = client['map_tile']
db.bay_area.drop()
bay_area = db['bay_area']
bay_area.insert_one(data)